In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np


In [ ]:
aqi = pd.read_csv("/content/aqi_data.csv")

aqi = aqi[['Year', 'State', 'Pop_Est', 'Good Days', 'Moderate Days',
           'Unhealthy for Sensitive Groups Days', 'Unhealthy Days',
           'Very Unhealthy Days', 'Hazardous Days', 'Median AQI']]

aqi['State'] = aqi['State'].str.strip().str.title()


In [ ]:
states = sorted(aqi['State'].unique())
years = sorted(aqi['Year'].unique())

np.random.seed(42)
records = []

for year in years:
    for state in states:
        base = 0.856
        region_adj = np.random.normal(0, 0.02)
        coverage = np.clip(base + region_adj, 0.75, 0.95)
        records.append([
            year,
            state,
            round(coverage * 100, 2),
            round((1 - coverage) * 100, 2)
        ])

access = pd.DataFrame(
    records,
    columns=['Year', 'State', 'Coverage (%)', 'No Coverage (%)']
)

In [ ]:
merged = pd.merge(aqi, access, on=['Year', 'State'], how='left')

In [ ]:
pollution_score = (
    merged['Unhealthy Days'] +
    merged['Very Unhealthy Days'] +
    merged['Hazardous Days']
)

merged['Pollution_Index'] = pollution_score / pollution_score.max()
merged['Access_Index'] = merged['Coverage (%)'] / 100

merged['Health_Risk_Index'] = (
    (1 - merged['Access_Index']) + merged['Pollution_Index']
)


In [ ]:
merged.to_csv("merged_health_env_dataset.csv", index=False)


In [ ]:
aqi = pd.read_csv("/content/aqi.csv")
access = pd.read_csv("/content/access.csv")

aqi = aqi[['Year', 'State', 'Pop_Est', 'Good Days', 'Moderate Days',
           'Unhealthy for Sensitive Groups Days', 'Unhealthy Days',
           'Very Unhealthy Days', 'Hazardous Days', 'Median AQI']]

access = access[['Year', 'State', 'Yes', 'No']]

aqi['State'] = aqi['State'].astype(str).str.strip().str.title()
access['State'] = access['State'].astype(str).str.strip().str.title()


In [ ]:
numeric_cols_aqi = ['Pop_Est', 'Good Days', 'Moderate Days',
                    'Unhealthy for Sensitive Groups Days',
                    'Unhealthy Days', 'Very Unhealthy Days',
                    'Hazardous Days', 'Median AQI']

aqi[numeric_cols_aqi] = aqi[numeric_cols_aqi].apply(
    pd.to_numeric, errors='coerce'
)

access[['Yes', 'No']] = (
    access[['Yes', 'No']]
    .replace('%', '', regex=True)
    .apply(pd.to_numeric, errors='coerce')
)


In [ ]:
merged = pd.merge(aqi, access, on=['Year', 'State'], how='left')

merged['Pollution_Index'] = (
    merged[['Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days']].sum(axis=1)
    / merged[['Good Days', 'Moderate Days',
              'Unhealthy for Sensitive Groups Days',
              'Unhealthy Days', 'Very Unhealthy Days',
              'Hazardous Days']].sum(axis=1)
).clip(0, 1)

merged['Access_Index'] = merged['Yes'] / 100


In [ ]:
missing_mask = merged['Median AQI'].isna() | merged['Pollution_Index'].isna()
state_mean = merged.groupby('State')['Median AQI'].transform('mean')

merged.loc[missing_mask, 'Median AQI'] = (
    state_mean[missing_mask] *
    (1.1 - merged.loc[missing_mask, 'Access_Index'].fillna(0.85))
)

merged['Median AQI'] = merged['Median AQI'].fillna(merged['Median AQI'].mean())

merged['Adjusted_Pollution'] = (
    merged['Pollution_Index'] * (1.1 - merged['Access_Index'])
)

merged['Env_Health_Burden'] = (
    merged['Adjusted_Pollution'] * 0.6 +
    (1 - merged['Access_Index']) * 0.4
).round(3)

merged.to_csv("merged_synthetic_env_health.csv", index=False)


In [ ]:
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
    "Connecticut", "Delaware", "District Of Columbia", "Florida", "Georgia",
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
    "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
    "New Hampshire", "New Jersey", "New Mexico", "New York",
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]

years = np.arange(1980, 2026)

full_grid = pd.MultiIndex.from_product(
    [states, years], names=['State', 'Year']
).to_frame(index=False)

merged_full = pd.merge(
    full_grid, merged, on=['State', 'Year'], how='left'
)


In [ ]:
health_events = pd.read_excel(
    "/content/drive/MyDrive/my_info_viz/ENGR-E 483_583_Health_Events_cleaned.xlsx"
)

health_events["Event_Date"] = pd.to_datetime(health_events["Event_Date"])
health_events["Event_Year"] = health_events["Event_Date"].dt.year

health_history = pd.read_excel(
    "/content/drive/MyDrive/my_info_viz/ENGR-E 483_583_Medical_History_final.xlsx"
)

merged = health_events.merge(
    health_history[["Medcal_Record_Number", "Diagnosed_date", "Diagnosis_Year"]],
    on="Medcal_Record_Number",
    how="left"
)

last_events = (
    merged.sort_values(["Medcal_Record_Number", "Event_Date"])
          .groupby("Medcal_Record_Number")
          .tail(1)
)

last_events["Longevity_Years"] = (
    (last_events["Event_Date"] - last_events["Diagnosed_date"]).dt.days / 365
)

last_events.loc[last_events["Outcome"] == "Ongoing",
                "Longevity_Years"] = (
                    2025 - last_events["Diagnosis_Year"]
                )

last_events.to_excel(
    "/content/drive/MyDrive/my_info_viz/Longevity_calculated.xlsx"
)


In [ ]:
patient_df = pd.read_excel(
    "/content/drive/MyDrive/my_info_viz/ENGR-E 483_583_Patients_Data.xlsx"
)

bool_summary = (
    patient_df.groupby("State")
              .agg(
                  Living_Alone_Count=("Living_Alone", "sum"),
                  Opioid_Addiction_Count=("Opioid_Addiction", "sum")
              )
)

alcohol_pivot = (
    patient_df.pivot_table(
        index="State",
        columns="Alcohol_Consumption",
        aggfunc="size",
        fill_value=0
    )
).rename(columns=lambda c: f"Alcohol_{c}")

smoking_pivot = (
    patient_df.pivot_table(
        index="State",
        columns="Smoking_Status",
        aggfunc="size",
        fill_value=0
    )
).rename(columns=lambda c: f"Smoking_{c}")

physical_pivot = (
    patient_df.pivot_table(
        index="State",
        columns="Physical_Activity_Level",
        aggfunc="size",
        fill_value=0
    )
).rename(columns=lambda c: f"Physical_{c}")

final_summary = (
    bool_summary
    .join(alcohol_pivot, how="left")
    .join(smoking_pivot, how="left")
    .join(physical_pivot, how="left")
    .reset_index()
)

final_summary.to_excel(
    "/content/drive/MyDrive/my_info_viz/personal_choices_patients.xlsx"
)


In [ ]:
env_data = pd.read_excel(
    "/content/drive/MyDrive/my_info_viz/statewise_data_health_env_cleaned.xlsx"
)

states_in_patients = patient_df["State"].unique()

env_filtered = env_data[env_data["State"].isin(states_in_patients)]

env_filtered.to_excel(
    "/content/drive/MyDrive/my_info_viz/filtered_env_health_cleaned.xlsx"
)
